In [20]:
from gene_level_MoM import *
import zipfile

In [7]:
###unzip file
zip_files = [
    "test_data/chr1_Contiguous1KSNP.zip",
    "test_data/chr1_Random1KSNP.zip"
]

for zf in zip_files:
    with zipfile.ZipFile(zf, "r") as z:
        z.extractall("test_data")   # 直接解压到 test_data/ 根目录
    print(f"Unzipped: {zf}")


Unzipped: test_data/chr1_Contiguous1KSNP.zip
Unzipped: test_data/chr1_Random1KSNP.zip


In [21]:
##read and process data
##random snp
RandomSNP = pd.read_csv("test_data/chr1_Random1KSNP.raw", sep=r'\s+')
RandomSNP = RandomSNP.iloc[:, 6:].to_numpy()
##Contiguous snp
ContiguousSNP = pd.read_csv("test_data/chr1_Contiguous1KSNP.raw", sep=r'\s+')
ContiguousSNP = ContiguousSNP.iloc[:, 6:].to_numpy()

In [22]:
##Get weight
X1 = ContiguousSNP[:2000,:200]
Z = (X1 - X1.mean(axis=0)) / X1.std(axis=0)

# Compute weights
w = weight_vector(X1)

print("\nWithout weights:")
print(check_weighted_products(Z, 1)[2])

print("\nWith weights:")
print(check_weighted_products(Z, w)[2])


Without weights:


NameError: name 'check_weighted_products' is not defined

In [ ]:
# Objective 1: Log-space 
def log_space_objective(u):
    total = 0
    for i in range(m):
        for j in range(i + 1, m):
            v = (Z[:, i] * Z[:, j]).var()
            c_ij = -0.5 * np.log(max(v, 1e-10))
            total += (u[i] + u[j] - c_ij) ** 2
    return total

# Objective 2: Original-space
def original_space_objective(u):
    w = np.exp(u)
    Zw = Z * w
    total = 0
    for i in range(m):
        for j in range(i + 1, m):
            v = (Zw[:, i] * Zw[:, j]).var()
            total += (v - 1) ** 2
    return total

u_unweighted = np.zeros(m)  # corresponds to w = 1

print("=== Log-space objective ===")
print(f"Unweighted: {log_space_objective(u_unweighted):.4f}")
print(f"LS weights: {log_space_objective(u_ls):.4f}")

print("\n=== Original-space objective ===")
print(f"Unweighted: {original_space_objective(u_unweighted):.4f}")
print(f"LS weights: {original_space_objective(u_ls):.4f}")